In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load and merge data from multiple sites
def load_and_merge_data(folder, site_names):
    dfs = []
    for site in site_names:
        file_path = f"{folder}PRSA_Data_{site}_20130301-20170228.csv"
        df = pd.read_csv(file_path)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Selected sites
selected_sites = ["Aotizhongxin", "Changping", "Dingling", "Dongsi", "Guanyuan"]
data_folder = "PRSA_Data_20130301-20170228/"

# Execute data loading
data = load_and_merge_data(data_folder, selected_sites)

In [2]:
print("\n=== Data Overview ===")
print(f"Shape: {data.shape}")
print("\nData Types:")
data.dtypes
print("\nMissing Values:")
(data.isnull().sum()/len(data)*100).round(2)


=== Data Overview ===
Shape: (175320, 18)

Data Types:

Missing Values:


No         0.00
year       0.00
month      0.00
day        0.00
hour       0.00
PM2.5      2.19
PM10       1.68
SO2        1.96
NO2        2.96
CO         5.85
O3         3.07
TEMP       0.09
PRES       0.09
DEWP       0.09
RAIN       0.09
wd         0.30
WSPM       0.07
station    0.00
dtype: float64